In [1]:
# Disabling warnings
# Отключение отображения предупреждений
import sys
import os
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

In [2]:
# Импорт библиотек
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime as dt
from datetime import time

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import recall_score, f1_score

from imblearn.over_sampling import RandomOverSampler

In [3]:
# Настройка pandas для того, чтобы отображались все колонки
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Значение порога мультиколлинеарности по умолчанию выбрано 0.6

Статья о мультиколлинеарнсоти (заключение авторов: мультиколлинеарность начинает ухудшать модель с порога 0.7
https://onlinelibrary.wiley.com/doi/full/10.1111/j.1600-0587.2012.07348.x

In [5]:
# Источник: stackoverflow
# Feature selection class to eliminate multicollinearity
# Выбор и удаление признаков скоррелированных между собой (борьба с мультиколлинеарносстью)
class MultiCollinearityEliminator():

    # Class initialisation
    # Инициализация класса
    def __init__(self, df, target, threshold=0.6):
        self.df = df
        self.target = target
        self.threshold = threshold

    # Method to create and return the feature correlation matrix dataframe
    # Метод создающий и возращающий корреляционную матрицу признаков датафрейма
    def createCorrMatrix(self, include_target=False):
        # Checking we should include the target in the correlation matrix
        if (include_target == False):
            df_temp = self.df.drop([self.target], axis=1)

            # Setting method to Pearson to prevent issues in case the default method for df.corr() gets changed
            # Setting min_period to 30 for the sample size to be statistically significant (normal) according to
            # central limit theorem
            corrMatrix = df_temp.corr(method='pearson', min_periods=30).abs()

        # Target is included for creating the series of feature to target correlation - Please refer the notes under the
        # print statement to understand why we create the series of feature to target correlation
        elif (include_target == True):
            corrMatrix = self.df.corr(method='pearson', min_periods=30).abs()

        return corrMatrix

    # Method to create and return the feature to target correlation matrix dataframe
    # Метод создающий и возвращающий корреляционную матрицу признаков с целевой переменной
    def createCorrMatrixWithTarget(self):

        # After obtaining the list of correlated features, this method will help to view which variables
        # (in the list of correlated features) are least correlated with the target
        # This way, out the list of correlated features, we can ensure to elimate the feature that is
        # least correlated with the target
        # This not only helps to sustain the predictive power of the model but also helps in reducing model complexity

        # Obtaining the correlation matrix of the dataframe (along with the target)
        corrMatrix = self.createCorrMatrix(include_target=True)

        # Creating the required dataframe, then dropping the target row
        # and sorting by the value of correlation with target (in asceding order)
        corrWithTarget = pd.DataFrame(corrMatrix.loc[:, self.target]).drop(
            [self.target], axis=0).sort_values(by=self.target)
        #print(corrWithTarget, '\n')
        return corrWithTarget

    # Method to create and return the list of correlated features
    # Метод создающий и вовзращающий лист скоррелированных признаков
    def createCorrelatedFeaturesList(self):
        # Obtaining the correlation matrix of the dataframe (without the target)
        corrMatrix = self.createCorrMatrix(include_target=False)
        colCorr = []
        # Iterating through the columns of the correlation matrix dataframe
        for column in corrMatrix.columns:
            # Iterating through the values (row wise) of the correlation matrix dataframe
            for idx, row in corrMatrix.iterrows():
                if(row[column] > self.threshold) and (row[column] < 1):
                    # Adding the features that are not already in the list of correlated features
                    if (idx not in colCorr):
                        colCorr.append(idx)
                    if (column not in colCorr):
                        colCorr.append(column)
        #print(colCorr, '\n')
        return colCorr

    # Method to eliminate the least important features from the list of correlated features
    # Метод удаляющий наименее важные признаки (наименее скоррелированные с целевым признаком) из двух скоррелированных признаков
    def deleteFeatures(self, colCorr):
        # Obtaining the feature to target correlation matrix dataframe
        corrWithTarget = self.createCorrMatrixWithTarget()
        for idx, row in corrWithTarget.iterrows():
            #print(idx, '\n')
            if (idx in colCorr):
                self.df = self.df.drop(idx, axis=1)
                break
        return self.df

    # Method to run automatically eliminate multicollinearity
    # Метод запускающий и удаляющий мультиколлинеарность
    def autoEliminateMulticollinearity(self):
        # Obtaining the list of correlated features
        colCorr = self.createCorrelatedFeaturesList()
        while colCorr != []:
            # Obtaining the dataframe after deleting the feature (from the list of correlated features)
            # that is least correlated with the taregt
            self.df = self.deleteFeatures(colCorr)
            # Obtaining the list of correlated features
            colCorr = self.createCorrelatedFeaturesList()
        return self.df

In [6]:
# Функция, возвращающая датафрейм, в котором исключены признаки имеющие корреляцию с целевым признаком
# менее заданного значения (по умолчанию: 0.05)
def reduce_low_corrs_with_target(df, target, threshold=0.05, return_predicted=False, X_test=None):
    # Маска True/False которая указывает какие признаки имеют корреляцию выше пороговой
    mask = df.corr().abs()[target].sort_values() > threshold
    # Возвращение датафрейма только с признаками, которые имеют корреляцию выше пороговой
    df = df[mask.index[mask]]
    # Для удаления лишних признаков из тестового набора данных:
    if return_predicted == True:
        # В маске для тестового датасета исключается таргет, так как его там попросту нет
        test_mask = mask.drop(target)
        # Тестовый набор без лишних признаков
        X_test = X_test[test_mask.index[test_mask]]
        # Возвращение тренировочного и тестового датасета
        return df, X_test
    else:
        return df

In [7]:
# Функция возвращающая значения 1 или 0, в зависимости от выполнения условия
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

In [8]:
# Функция, возвращающая оптимальный порог отнесения к классу (в данной функции - по метрике recall_macro)
def find_opt_threshold(y_valid, probs):
    # вероятности только для положительного класса (1)
    probs = probs[:, 1]
    # определение порогов
    thresholds = np.arange(0, 1, 0.001)
    # подсчет метрики для каждого порога
    scores = [recall_score(y_valid, to_labels(probs, t),
                           average='macro') for t in thresholds]
    # получение наилучшего порогового значнеия 
    ix = np.argmax(scores)
    best_threshold = thresholds[ix]
    print('Threshold=%.3f, Recall_macro=%.5f' % (thresholds[ix], scores[ix]))
    return best_threshold

## Описание признаков и последующих замен
| Исходный признак | Замена | Описание |
| --- | --- | --- |
|'ID'|***'id'***| идентификатор опрошенного|
|'Пол' | ***'sex'*** | пол |   
|'Семья' | ***'family'*** | семейное положение|
|'Этнос' | ***'ethnos'*** | этнос  |
|'Национальность' | '***'nationality'*** | национальность  |
|'Религия' | ***'religion'*** | отношение к религиозной группе  |
|'Образование' | ***''education'***' | уровень образования  |
|'Профессия' | ***''profession'***' | профессия  |
|'Вы работаете?' | ***'job'*** | наличие работы в данный момент  |
|'Выход на пенсию' | ***'retire'*** | является ли опрошенный пенсионером  |
|'Прекращение работы по болезни | ***'stop_work_due_disease'*** | связано ли прекращение работы с болезнью  |
|'Сахарный диабет' | ***'diabetes'*** | наличие сахарного диабета (любого типа)  |
|'Гепатит' | ***'hepatitis'*** | наличие гепатита  |
|'Онкология' | ***'oncology'*** | наличие онкологических заболеваний  |
|'Хроническое заболевание легких' | ***'chronic_lung_disease'*** | наличие хронического заболевания легких  |
|'Бронжиальная астма' | ***'bronchial_asthma'*** | наличие бронхиальной астмы  |
|'Туберкулез легких ' | ***'tuberculosis'*** | наличие туберкулеза  |
|'ВИЧ/СПИД' |  ***'hiv/aids'*** | наличие ВИЧ/СПИД  |
|'Регулярный прим лекарственных средств' | ***'intake_medicines'*** | факт регулярного приема лекарственных средств | 
|'Травмы за год' | ***'trauma_last_year'*** | факт получения травм за последний год  |
|'Переломы' | ***'fractures'*** | наличие переломов  |
|'Статус Курения' | ***'smoking'*** | статус курения в данный момент  |
|'Возраст курения' | ***'smoking_duration'*** | длительность курения  |
|'Сигарет в день' | ***'ciggaretes_per_day'*** | количество сигарет в день  |
|'Пассивное курение' | ***'passive_smoking'*** | наличие курящих в окружении  |
|'Частота пасс кур' | ***'passive_smoking_frequency'*** | частота нахождения среди курящих  |
|'Алкоголь' | ***'alcohol'*** | отношение к приему алкоголя  |
|'Возраст алког' | ***'alcohol_duration'*** | длительность употребления алкоголя (годы)  |
|'Время засыпания' | ***'time_sleep_onset'*** | время засыпания  |
|'Время пробуждения' | ***'time_sleep_upset'*** | время пробуждения  |
|'Сон после обеда' | ***'midday_sleep'*** | сон в середине дня (после обеда)  |
|'Спорт, клубы' | ***'sport'*** | факт регулярного занятия спортом(фитнесом)  |
|'Религия, клубы' | ***'religion_clubs'*** | факт регулярного посещения религиозных мероприятий, групп  |
|'ID_y' | ***'id_y'*** | идентификатор (связь с таблицей диагнозов)  |
|'Артериальная гипертензия' | ***'arterial_hypertension'*** | наличие артериальной гипертензии  |
|'ОНМК' | ***'stroke'*** | наличие острого нарушения мозгового кровообращения (инсульта)  |
|'Стенокардия, ИБС, инфаркт миокарда' | ***'IHD'*** | наличие ишемической болезни сердца (все формы)  |
|'Сердечная недостаточность' | ***'heart_failure'*** | наличие сердечной недостаточности  |
|'Прочие заболевания сердца' | ***'other_cardio_diseases'*** | наличие других сердечных заболеваний  |

# Подготовка и очистка данных 

In [9]:
df_train = pd.read_csv('data/train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955 entries, 0 to 954
Data columns (total 39 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ID                                     955 non-null    object 
 1   Пол                                    954 non-null    object 
 2   Семья                                  955 non-null    object 
 3   Этнос                                  955 non-null    object 
 4   Национальность                         955 non-null    object 
 5   Религия                                955 non-null    object 
 6   Образование                            955 non-null    object 
 7   Профессия                              955 non-null    object 
 8   Вы работаете?                          955 non-null    int64  
 9   Выход на пенсию                        955 non-null    int64  
 10  Прекращение работы по болезни          955 non-null    int64  
 11  Сахарн

In [10]:
df_test = pd.read_csv('data/test_dataset_test.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 33 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ID                                     638 non-null    object 
 1   Пол                                    638 non-null    object 
 2   Семья                                  638 non-null    object 
 3   Этнос                                  638 non-null    object 
 4   Национальность                         638 non-null    object 
 5   Религия                                638 non-null    object 
 6   Образование                            638 non-null    object 
 7   Профессия                              638 non-null    object 
 8   Вы работаете?                          638 non-null    int64  
 9   Выход на пенсию                        638 non-null    int64  
 10  Прекращение работы по болезни          638 non-null    int64  
 11  Сахарн

In [11]:
submit = pd.read_csv('data/sample_solution.csv')
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   ID                                  638 non-null    object
 1   Артериальная гипертензия            638 non-null    int64 
 2   ОНМК                                638 non-null    int64 
 3   Стенокардия, ИБС, инфаркт миокарда  638 non-null    int64 
 4   Сердечная недостаточность           638 non-null    int64 
 5   Прочие заболевания сердца           638 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 30.0+ KB


In [12]:
# Для корректной обработки признаков объединяем тренировочный и тестовый набор данных в один датасет

# выделение целевых значений и датасета
target_train = df_train.iloc[:, -5:]
df_train = df_train.iloc[:, :-5]

# вспомогательный флаг тренировочного и тестовой частей общего набора данных
df_train['train'] = 1
df_test['train'] = 0


# объединение в один датасет
df = pd.concat([df_train, df_test]).reset_index(drop=True)

In [13]:
# Функция, которая выводит корреляцию признака с целевыми переменными (содержатся в разных датафреймах)
def check_corrs_with_target(df, target_train=target_train, col=df.columns.to_list()):
    df_train = df[df.train == 1]
    df_for_check = pd.concat([df_train, target_train], axis=1)
    cols_target = target_train.columns.to_list()
    # col.extend(cols_target)
    print('corrs_matrix_between: ', col)
    corrs = df_for_check.corr()
    return corrs.loc[col, cols_target]

In [14]:
df.head(1)

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,train
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,54-102-358-02,1


In [15]:
feature_map = {
    'ID': 'id',
    'Пол': 'sex',
    'Семья': 'family',
    'Этнос': 'ethnos',
    'Национальность': 'nationality',
    'Религия': 'religion',
    'Образование': 'education',
    'Профессия': 'profession',
    'Вы работаете?': 'job',
    'Выход на пенсию': 'retired',
    'Прекращение работы по болезни': 'stop_work_due_disease',
    'Сахарный диабет': 'diabetes',
    'Гепатит': 'hepatitis',
    'Онкология': 'oncology',
    'Хроническое заболевание легких': 'chronic_lung_disease',
    'Бронжиальная астма': 'bronchial_asthma',
    'Туберкулез легких ': 'tuberculosis',
    'ВИЧ/СПИД': 'hiv/aids',
    'Регулярный прим лекарственных средств': 'intake_medicines',
    'Травмы за год': 'trauma_last_year',
    'Переломы': 'fractures',
    'Статус Курения': 'smoking',
    'Возраст курения': 'smoking_duration',
    'Сигарет в день': 'ciggaretes_per_day',
    'Пассивное курение': 'passive_smoking',
    'Частота пасс кур': 'passive_smoking_frequency',
    'Алкоголь': 'alcohol',
    'Возраст алког': 'alcohol_duration',
    'Время засыпания': 'time_sleep_onset',
    'Время пробуждения': 'time_sleep_upset',
    'Сон после обеда': 'midday_sleep',
    'Спорт, клубы': 'sport',
    'Религия, клубы': 'religion_clubs',
    'ID_y': 'id_y',
    'Артериальная гипертензия': 'arterial_hypertension',
    'ОНМК': 'stroke',
    'Стенокардия, ИБС, инфаркт миокарда': 'IHD',
    'Сердечная недостаточность': 'heart_failure',
    'Прочие заболевания сердца': 'other_cardio_diseases'}

In [16]:
df.rename(feature_map, axis=1, inplace=True)

In [17]:
df.columns

Index(['id', 'sex', 'family', 'ethnos', 'nationality', 'religion', 'education',
       'profession', 'job', 'retired', 'stop_work_due_disease', 'diabetes',
       'hepatitis', 'oncology', 'chronic_lung_disease', 'bronchial_asthma',
       'tuberculosis', 'hiv/aids', 'intake_medicines', 'trauma_last_year',
       'fractures', 'smoking', 'smoking_duration', 'ciggaretes_per_day',
       'passive_smoking', 'passive_smoking_frequency', 'alcohol',
       'alcohol_duration', 'time_sleep_onset', 'time_sleep_upset',
       'midday_sleep', 'sport', 'religion_clubs', 'id_y', 'train'],
      dtype='object')

In [18]:
target_train.rename(feature_map, axis=1, inplace=True)

In [19]:
target_train.columns

Index(['arterial_hypertension', 'stroke', 'IHD', 'heart_failure',
       'other_cardio_diseases'],
      dtype='object')

In [20]:
# Проверка на совпадение ID в train
print((df[df.train == 1].id == df[df.train == 1].id_y).sum() /
      len(df[df.train == 1]))
# Проверка на совпадение ID в test и submit
print((df[df.train == 0].id.reset_index(
    drop=True) == submit.ID).sum() / len(submit))

1.0
1.0


In [21]:
df.id[:10]

0    54-102-358-02
1    54-103-101-01
2    54-501-026-03
3    54-501-094-02
4    54-503-022-01
5    54-002-074-01
6    54-602-027-01
7    54-102-072-01
8    54-102-145-01
9    54-002-002-01
Name: id, dtype: object

# ADD FEATURE unknown_id_feature, id_1, id_2, id_3, id_4

Последние цифры id могут характеризовать все что угодно (возможно, не имеют ценности)
Предположительно это могут быть:
1. Место проведение опроса (улица, поликлиника, больница, общественное место) 
2. Источник опроса (опросил сотрудник, бумажный носитель, интернет-анкета и др. 
3. id сотрудника, проводившего опрос  

Место и источник могут влиять на результат опроса


In [22]:
# df['unknown_id_feature_1'] = df['id'].apply(lambda x: int(x[0:2]))
# df['unknown_id_feature_2'] = df['id'].apply(lambda x: int(x[3:6]))
# df['unknown_id_feature_3'] = df['id'].apply(lambda x: int(x[7:10]))


df['unknown_id_feature'] = df['id'].apply(lambda x: int(x[-1:]))


id_features = pd.get_dummies(df['unknown_id_feature'], prefix='id')

df = pd.concat([df, id_features], axis=1)

In [23]:
df.drop(['id', 'id_y'], axis=1, inplace=True)

In [24]:
# total amount of missing data
# Общее количество пропусков в данных
df.isna().sum()

sex                             1
family                          0
ethnos                          0
nationality                     0
religion                        0
education                       0
profession                      0
job                             0
retired                         0
stop_work_due_disease           0
diabetes                        0
hepatitis                       0
oncology                        0
chronic_lung_disease            0
bronchial_asthma                0
tuberculosis                    0
hiv/aids                        0
intake_medicines                0
trauma_last_year                0
fractures                       0
smoking                         0
smoking_duration              912
ciggaretes_per_day            919
passive_smoking                 0
passive_smoking_frequency    1216
alcohol                         0
alcohol_duration              304
time_sleep_onset                0
time_sleep_upset                0
midday_sleep  

### Колонка sex

In [25]:
df.sex.value_counts(dropna=False)

Ж      1117
М       475
NaN       1
Name: sex, dtype: int64

In [26]:
# Заполнение пропуска модой (в данном случае - Ж)
df.sex.fillna(df.sex.mode()[0], inplace=True)

In [27]:
df.sex.value_counts(dropna=False)

Ж    1118
М     475
Name: sex, dtype: int64

In [28]:
df.sex = np.where(df.sex == 'Ж', 0, 1)

In [29]:
# Оценка корреляции с целевыми признаками
check_corrs_with_target(df, col=['sex'])

corrs_matrix_between:  ['sex']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
sex,-0.119174,0.075247,0.005842,-0.08241,-0.086526


### Колонка family

In [30]:
df.family.value_counts(dropna=False)

в браке в настоящее время                          937
вдовец / вдова                                     246
в разводе                                          201
гражданский брак / проживание с партнером          129
никогда не был(а) в браке                           77
раздельное проживание (официально не разведены)      3
Name: family, dtype: int64

In [31]:
# Создание отдельного датафрейма со статусами семейного положение
family = pd.get_dummies(df.family)
print('before replace: ', list(family.columns))

# Словарь для замены названий колонок
fam_cols_map = {'в браке в настоящее время': 'married',
                'в разводе': 'divorced',
                'вдовец / вдова': 'widowed',
                'гражданский брак / проживание с партнером': 'cohabited',
                'никогда не был(а) в браке': 'single',
                'раздельное проживание (официально не разведены)': 'separated'}

# Замена названий колонок в датафрейме
family.rename(fam_cols_map, axis=1, inplace=True)
print('after replace: ', list(family.columns))

before replace:  ['в браке в настоящее время', 'в разводе', 'вдовец / вдова', 'гражданский брак / проживание с партнером', 'никогда не был(а) в браке', 'раздельное проживание (официально не разведены)']
after replace:  ['married', 'divorced', 'widowed', 'cohabited', 'single', 'separated']


In [32]:
# Добавление статусов о семейном положении в основной датафрейм
df = pd.concat([df, family], axis=1)
df.drop('family', axis=1, inplace=True)

In [33]:
# Матрица корреляций статусов семейного положения и целевых переменных
fam_cols = family.columns
check_corrs_with_target(df, col=fam_cols)

corrs_matrix_between:  Index(['married', 'divorced', 'widowed', 'cohabited', 'single', 'separated'], dtype='object')


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
married,-0.080766,0.019210,-0.025806,-0.046685,-0.027510
divorced,-0.014791,0.009703,-0.040850,-0.005838,0.029754
widowed,0.189502,0.012461,0.093806,0.132974,0.032011
cohabited,-0.037290,-0.026097,-0.019460,-0.049826,-0.001516
single,-0.051434,-0.045989,-0.004129,-0.039003,-0.033028
separated,-0.042882,-0.009703,-0.017117,-0.015315,-0.014411


### Колонка ethnos

In [34]:
df.ethnos.value_counts(dropna=False)

европейская                                                                                                      1559
другая азиатская (Корея, Малайзия, Таиланд, Вьетнам, Казахстан, Киргизия, Туркмения, Узбекистан, Таджикистан)      24
прочее (любая иная этно-расовая группа, не представленная выше)                                                    10
Name: ethnos, dtype: int64

In [35]:
ethnos = pd.get_dummies(df.ethnos)
print('before replace: ', list(ethnos.columns))

ethnos_col_map = {'другая азиатская (Корея, Малайзия, Таиланд, Вьетнам, Казахстан, Киргизия, Туркмения, Узбекистан, Таджикистан)': 'asian',
                  'европейская': 'european',
                  'прочее (любая иная этно-расовая группа, не представленная выше)': 'other'}

ethnos.rename(ethnos_col_map, axis=1, inplace=True)

print('after replace: ', list(ethnos.columns))

before replace:  ['другая азиатская (Корея, Малайзия, Таиланд, Вьетнам, Казахстан, Киргизия, Туркмения, Узбекистан, Таджикистан)', 'европейская', 'прочее (любая иная этно-расовая группа, не представленная выше)']
after replace:  ['asian', 'european', 'other']


In [36]:
df = pd.concat([df, ethnos], axis=1)
df.drop('ethnos', axis=1, inplace=True)

In [37]:
ethnos_col = ethnos.columns
check_corrs_with_target(df, col=ethnos_col)

corrs_matrix_between:  Index(['asian', 'european', 'other'], dtype='object')


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
asian,0.000964,-0.028513,0.022155,0.007666,-0.042351
european,0.002794,0.034006,-0.001218,0.009179,0.050509
other,-0.006621,-0.018200,-0.032108,-0.028727,-0.027032


### Колонка nationality

In [38]:
df.nationality.value_counts(dropna=False)

Русские                  1500
Татары                     33
Немцы                      10
Украинцы                   10
Чуваши                      9
Азербайджанцы               4
Белорусы                    3
Другие национальности       3
Казахи                      3
Армяне                      3
Удмурты                     2
Евреи                       2
Таджики                     2
Киргизы                     2
Мордва                      2
Башкиры                     1
Молдаване                   1
Буряты                      1
Эстонцы                     1
Лезгины                     1
Name: nationality, dtype: int64

In [39]:
# Добавление колонки is_russian, так как количество других национальностей слишком мало
df['is_russian'] = np.where(df['nationality'] == 'Русские', 1, 0)

In [40]:
# Удаление исходной колонки
df.drop('nationality', axis=1, inplace=True)

In [41]:
check_corrs_with_target(df, col=['is_russian'])

corrs_matrix_between:  ['is_russian']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
is_russian,-0.016501,0.008887,-0.056265,-0.035137,0.016238


### Колонка religion

In [42]:
df.religion.value_counts(dropna=False)

Христианство         1361
Атеист / агностик     127
Нет                    73
Ислам                  30
Другое                  1
Индуизм                 1
Name: religion, dtype: int64

In [43]:
religion = pd.get_dummies(df.religion)
print('before replace: ', list(religion.columns))

religion_cols_map = {'Атеист / агностик': 'atheist',
                     'Другое': 'other',
                     'Индуизм': 'hinduist',
                     'Ислам': 'muslim',
                     'Нет': 'no',
                     'Христианство': 'christian'}


religion.rename(religion_cols_map, axis=1, inplace=True)
print('after replace: ', list(religion.columns))

before replace:  ['Атеист / агностик', 'Другое', 'Индуизм', 'Ислам', 'Нет', 'Христианство']
after replace:  ['atheist', 'other', 'hinduist', 'muslim', 'no', 'christian']


In [44]:
df = pd.concat([df, religion], axis=1)

In [45]:
religion_cols = religion.columns
check_corrs_with_target(df, col=religion_cols)

corrs_matrix_between:  Index(['atheist', 'other', 'hinduist', 'muslim', 'no', 'christian'], dtype='object')


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
atheist,-0.045189,-0.030536,-0.017138,-0.032177,-0.035071
other,-0.006621,-0.018200,-0.032108,-0.028727,-0.027032
other,NaN,NaN,NaN,NaN,NaN
hinduist,NaN,NaN,NaN,NaN,NaN
muslim,-0.016203,-0.034006,0.001218,-0.009179,-0.050509
no,-0.011883,-0.019373,0.015392,0.015093,-0.012490
christian,0.049008,0.049311,0.004372,0.020798,0.056125


In [46]:
# Ввиду отсутствия значимых корреляций, решено оставить только колонку christian.
df.drop(['atheist', 'hinduist', 'muslim', 'no', 'other'], axis=1, inplace=True)

In [47]:
df.drop('religion', axis=1, inplace=True)

In [48]:
check_corrs_with_target(df, col='christian')

corrs_matrix_between:  christian


arterial_hypertension    0.049008
stroke                   0.049311
IHD                      0.004372
heart_failure            0.020798
other_cardio_diseases    0.056125
Name: christian, dtype: float64

### Колонка education

In [49]:
df.education.value_counts(dropna=False)

4 - профессиональное училище                         757
5 - ВУЗ                                              547
3 - средняя школа / закон.среднее / выше среднего    279
2 - начальная школа                                   10
Name: education, dtype: int64

In [50]:
# Приведение к порядковому признаку начинающемуся с 1
df.education = df.education.apply(lambda x: int(x[0])-1)

In [51]:
check_corrs_with_target(df, col='education')

corrs_matrix_between:  education


arterial_hypertension   -0.106846
stroke                  -0.064249
IHD                     -0.050195
heart_failure           -0.080554
other_cardio_diseases    0.016931
Name: education, dtype: float64

### Колонка profession

In [52]:
df.profession.value_counts(dropna=False)

дипломированные специалисты                                                            382
низкоквалифицированные работники                                                       225
работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков        205
ремесленники и представители других отраслей промышленности                            149
служащие                                                                               142
техники и младшие специалисты                                                          140
ведение домашнего хозяйства                                                            133
операторы и монтажники установок и машинного оборудования                              128
представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры     55
квалифицированные работники сельского хозяйства и рыболовного                           25
вооруженные силы                                                                         9

In [53]:
profession = pd.get_dummies(df.profession)
profession.columns

Index(['ведение домашнего хозяйства', 'вооруженные силы',
       'дипломированные специалисты',
       'квалифицированные работники сельского хозяйства и рыболовного',
       'низкоквалифицированные работники',
       'операторы и монтажники установок и машинного оборудования',
       'представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры',
       'работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков',
       'ремесленники и представители других отраслей промышленности',
       'служащие', 'техники и младшие специалисты'],
      dtype='object')

In [54]:
profession_cols_map = {'ведение домашнего хозяйства': 'household',
                       'вооруженные силы': 'military',
                       'дипломированные специалисты': 'senior_specialist',
                       'квалифицированные работники сельского хозяйства и рыболовного': 'farmer/fisherman',
                       'низкоквалифицированные работники': 'low_qualified',
                       'операторы и монтажники установок и машинного оборудования': 'hard_industry',
                       'представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры': 'top_management',
                       'работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков': 'service',
                       'ремесленники и представители других отраслей промышленности': 'crafters_other',
                       'служащие': 'office',
                       'техники и младшие специалисты': 'junior_specialists'}

profession.rename(profession_cols_map, axis=1, inplace=True)

In [55]:
df = pd.concat([df, profession], axis=1)
df.drop('profession', axis=1, inplace=True)

In [56]:
profession_cols = profession.columns
check_corrs_with_target(df, col=profession_cols)

corrs_matrix_between:  Index(['household', 'military', 'senior_specialist', 'farmer/fisherman',
       'low_qualified', 'hard_industry', 'top_management', 'service',
       'crafters_other', 'office', 'junior_specialists'],
      dtype='object')


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
household,0.061559,0.009688,0.058190,0.098212,-0.016993
military,0.019338,-0.015365,-0.027108,-0.024253,0.027865
senior_specialist,-0.049845,-0.020203,-0.004255,-0.037893,0.049462
farmer/fisherman,0.064455,0.049622,0.070460,0.060338,0.012978
low_qualified,-0.008645,-0.028277,0.009930,-0.048256,-0.056464
hard_industry,-0.092412,0.017783,-0.022026,-0.016324,-0.089831
top_management,-0.015030,-0.041310,-0.055884,-0.046672,-0.041891
service,-0.009041,-0.019749,-0.057863,0.037795,0.031926
crafters_other,0.060903,0.109801,0.012044,-0.047276,0.012399
office,-0.021777,0.002407,-0.000286,0.023276,0.036250


### Колонка job

In [57]:
df.job.value_counts(dropna=False)

1    865
0    728
Name: job, dtype: int64

In [58]:
check_corrs_with_target(df, col='job')

corrs_matrix_between:  job


arterial_hypertension   -0.224856
stroke                  -0.077221
IHD                     -0.206260
heart_failure           -0.176134
other_cardio_diseases   -0.058784
Name: job, dtype: float64

### Колонка retired

In [59]:
df.retired.value_counts(dropna=False)

0    1089
1     504
Name: retired, dtype: int64

In [60]:
check_corrs_with_target(df, col='retired')

corrs_matrix_between:  retired


arterial_hypertension    0.255916
stroke                   0.057585
IHD                      0.221894
heart_failure            0.175824
other_cardio_diseases    0.032419
Name: retired, dtype: float64

### Колонка stop_work_due_disease

In [61]:
df.stop_work_due_disease.value_counts(dropna=False)

0    1509
1      84
Name: stop_work_due_disease, dtype: int64

In [62]:
check_corrs_with_target(df, col='stop_work_due_disease')

corrs_matrix_between:  stop_work_due_disease


arterial_hypertension    0.044279
stroke                   0.097109
IHD                      0.080011
heart_failure            0.054904
other_cardio_diseases    0.031735
Name: stop_work_due_disease, dtype: float64

### Колонка diabetes

In [63]:
df.diabetes.value_counts(dropna=False)

0    1441
1     152
Name: diabetes, dtype: int64

### Колонка hepatitis

In [64]:
df.hepatitis.value_counts(dropna=False)

0    1410
1     183
Name: hepatitis, dtype: int64

### Колонка oncology

In [65]:
df.oncology.value_counts(dropna=False)

0    1521
1      72
Name: oncology, dtype: int64

### Колонка chronic_lung_disease

In [66]:
df.chronic_lung_disease.value_counts(dropna=False)

0    1469
1     124
Name: chronic_lung_disease, dtype: int64

### Колонка bronchial_asthma

In [67]:
df.bronchial_asthma.value_counts(dropna=False)

0    1530
1      63
Name: bronchial_asthma, dtype: int64

### Колонка tuberculosis

In [68]:
df.tuberculosis.value_counts(dropna=False)

0    1565
1      28
Name: tuberculosis, dtype: int64

### Колонка hiv/aids

In [69]:
df['hiv/aids'].value_counts(dropna=False)

0    1591
1       2
Name: hiv/aids, dtype: int64

In [70]:
# Колонки связанные с коморбидностью
comorbid_cols = ['diabetes', 'hepatitis', 'oncology',
                 'chronic_lung_disease', 'bronchial_asthma', 'hiv/aids']

In [71]:
check_corrs_with_target(df, col=comorbid_cols)

corrs_matrix_between:  ['diabetes', 'hepatitis', 'oncology', 'chronic_lung_disease', 'bronchial_asthma', 'hiv/aids']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
diabetes,0.230891,0.044747,0.141505,0.100181,0.034551
hepatitis,-0.030354,-0.014875,-0.020499,0.004706,-0.004002
oncology,0.090267,0.003835,0.042075,0.044964,0.055175
chronic_lung_disease,0.066575,-0.001782,0.002166,0.049622,0.062571
bronchial_asthma,0.091658,-0.044858,0.046903,0.100989,0.023601
hiv/aids,-0.042882,-0.009703,-0.017117,-0.015315,-0.014411


# ADD FEATURE (comorbid_count & is_comorbid)

In [72]:
# Количество опрошенных с определенными заболеваниями
df[comorbid_cols].apply(lambda x: sum(x))

diabetes                152
hepatitis               183
oncology                 72
chronic_lung_disease    124
bronchial_asthma         63
hiv/aids                  2
dtype: int64

In [73]:
df['comorbid_count'] = df[comorbid_cols].sum(axis=1)
df['is_comorbid'] = np.where(df['comorbid_count'] > 0, 1, 0)

In [74]:
check_corrs_with_target(df, col=['comorbid_count', 'is_comorbid'])

corrs_matrix_between:  ['comorbid_count', 'is_comorbid']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
comorbid_count,0.179487,-0.000460,0.086354,0.117311,0.065174
is_comorbid,0.178101,0.022106,0.095233,0.116283,0.084209


### Колонка intake_medicines

In [75]:
df.intake_medicines.value_counts(dropna=False)

1    995
0    598
Name: intake_medicines, dtype: int64

In [76]:
check_corrs_with_target(df, col='intake_medicines')

corrs_matrix_between:  intake_medicines


arterial_hypertension    0.452289
stroke                   0.114712
IHD                      0.218472
heart_failure            0.176454
other_cardio_diseases    0.119922
Name: intake_medicines, dtype: float64

### Колонка trauma_last_year

In [77]:
df.trauma_last_year.value_counts(dropna=False)

0    1508
1      85
Name: trauma_last_year, dtype: int64

In [78]:
check_corrs_with_target(df, col='trauma_last_year')

corrs_matrix_between:  trauma_last_year


arterial_hypertension    0.011036
stroke                  -0.004355
IHD                     -0.017729
heart_failure           -0.001962
other_cardio_diseases    0.039166
Name: trauma_last_year, dtype: float64

### Колонка fractures

In [79]:
df.fractures.value_counts(dropna=False)

0    1032
1     561
Name: fractures, dtype: int64

In [80]:
check_corrs_with_target(df, col='fractures')

corrs_matrix_between:  fractures


arterial_hypertension    0.097213
stroke                   0.070100
IHD                      0.077409
heart_failure            0.058426
other_cardio_diseases    0.034900
Name: fractures, dtype: float64

# ADD FEATURES (fracture_last_year, trauma_on_retire, fracture_on_retire)

In [81]:
# Признак о переломе кости за последний год (предположительно)
df['fracture_last_year'] = df['trauma_last_year'] * df['fractures']
# Признак о наличии травмы за последний год будучи в пенсионном возрасте
df['trauma_on_retire'] = df['trauma_last_year'] * df['retired']
# Признак о переломе кости за последний год (предположительном) будучи на пенсии
df['fracture_on_retire'] = df['fracture_last_year'] * df['retired']

In [82]:
trauma_features = ['fracture_last_year',
                   'trauma_on_retire', 'fracture_on_retire']
check_corrs_with_target(df, col=trauma_features)

corrs_matrix_between:  ['fracture_last_year', 'trauma_on_retire', 'fracture_on_retire']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
fracture_last_year,-0.003860,0.013674,-0.004894,-0.009606,0.016512
trauma_on_retire,0.032709,0.010554,0.046307,0.034002,0.040642
fracture_on_retire,0.006801,0.028961,0.055685,0.034031,0.039511


Примечание: генерирование вышеуказанных признаков связано с повышенными рисками развития заболеваний (не только сердечно-сосудистой системы) при переломе шейки бедра в пенсионном возрасте.

### Колонка smoking

In [83]:
df.smoking.value_counts(dropna=False)

Никогда не курил(а)    911
Курит                  360
Бросил(а)              321
Никогда не курил         1
Name: smoking, dtype: int64

In [84]:
# Исправление ошибки
df.smoking.replace('Никогда не курил', 'Никогда не курил(а)', inplace=True)

In [85]:
smoking = pd.get_dummies(df.smoking)
print('before replace: ', list(smoking.columns))

smoking_cols_map = {'Бросил(а)': 'previous_smoke',
                    'Курит': 'smoke', 'Никогда не курил(а)': 'never_smoke'}

smoking.rename(smoking_cols_map, axis=1, inplace=True)

print('after replace: ', list(smoking.columns))

before replace:  ['Бросил(а)', 'Курит', 'Никогда не курил(а)']
after replace:  ['previous_smoke', 'smoke', 'never_smoke']


In [86]:
df = df.drop('smoking', axis=True)
df = pd.concat([df, smoking], axis=1)

# ADD FEATURE smoker_score

In [87]:
df['smoker_score'] = df['never_smoke']*0 + \
    df['previous_smoke'] + df['smoke'] * 2

In [88]:
smoking_cols = ['previous_smoke', 'never_smoke', 'smoke', 'smoker_score']

In [89]:
check_corrs_with_target(df, col=smoking_cols)

corrs_matrix_between:  ['previous_smoke', 'never_smoke', 'smoke', 'smoker_score']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
previous_smoke,0.014692,-0.054241,0.020759,-0.045270,-0.001829
never_smoke,0.069542,-0.003255,0.015962,0.094327,0.008136
smoke,-0.095603,0.055267,-0.038434,-0.067838,-0.007820
smoker_score,-0.090244,0.030081,-0.029108,-0.090927,-0.008845


### Колонка smoking_duration

In [90]:
df.smoking_duration.value_counts(dropna=False)

NaN     912
20.0    111
18.0     94
16.0     72
17.0     66
15.0     51
19.0     33
25.0     31
30.0     23
14.0     18
12.0     15
21.0     15
22.0     14
7.0      13
24.0     11
40.0     11
23.0     10
13.0      9
10.0      9
35.0      9
6.0       5
27.0      5
45.0      5
26.0      5
8.0       4
28.0      4
29.0      4
33.0      3
36.0      3
41.0      3
50.0      3
31.0      2
39.0      2
47.0      2
52.0      1
44.0      1
38.0      1
32.0      1
56.0      1
42.0      1
55.0      1
58.0      1
11.0      1
51.0      1
34.0      1
37.0      1
53.0      1
5.0       1
54.0      1
43.0      1
Name: smoking_duration, dtype: int64

In [91]:
df['smoking_duration_score'] = df['smoking_duration'] // 5

In [92]:
df['smoking_duration_score'].value_counts(dropna=False)

NaN     912
3.0     316
4.0     161
2.0      52
5.0      49
6.0      30
1.0      23
8.0      17
7.0      16
10.0      7
9.0       7
11.0      3
Name: smoking_duration_score, dtype: int64

In [93]:
df['smoking_duration'].fillna(0, inplace=True)

In [94]:
df['smoking_duration_score'].fillna(0, inplace=True)

In [95]:
df['smoking_duration_score'] = np.where(
    df['smoking_duration_score'] > 9, 9, df['smoking_duration_score'])

In [96]:
check_corrs_with_target(df, col=['smoking_duration', 'smoking_duration_score'])

corrs_matrix_between:  ['smoking_duration', 'smoking_duration_score']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
smoking_duration,-0.036623,0.003147,-0.016511,-0.064340,0.020556
smoking_duration_score,-0.033445,0.000033,-0.017016,-0.059927,0.021710


### Колонка ciggaretes_per_day

In [97]:
df['ciggaretes_per_day_log'] = np.log(df.ciggaretes_per_day + 1)

In [98]:
df['ciggaretes_per_day_score'] = (df.ciggaretes_per_day // 10) + 1

In [99]:
df.ciggaretes_per_day_score = np.where(
    df.ciggaretes_per_day_score > 5, 5, df.ciggaretes_per_day_score)

In [100]:
ciggaretes_features = ['ciggaretes_per_day', 'ciggaretes_per_day_score',
                       'ciggaretes_per_day_log']

In [101]:
df['ciggaretes_per_day_score'].fillna(0, inplace=True)

In [102]:
for col in ciggaretes_features:
    df[col].fillna(0, inplace=True)

In [103]:
check_corrs_with_target(df, col=ciggaretes_features)

corrs_matrix_between:  ['ciggaretes_per_day', 'ciggaretes_per_day_score', 'ciggaretes_per_day_log']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
ciggaretes_per_day,-0.029327,0.007107,0.015671,-0.069713,-0.041501
ciggaretes_per_day_score,-0.045596,0.011395,0.013145,-0.078645,-0.042002
ciggaretes_per_day_log,-0.057114,0.010710,0.003519,-0.088335,-0.026594


# ADD FEATURE variants of smoking_score_int

In [104]:
df['smoking_score_int_log'] = (
    df['smoker_score'] * df['smoking_duration_score']) ** df['ciggaretes_per_day_log']

df['smoking_score_int_score'] = (
    df['smoker_score'] * df['smoking_duration_score']) ** df['ciggaretes_per_day_score']

df['smoking_score_int_score_2'] = (
    df['smoker_score'] * df['ciggaretes_per_day_score']) * df['smoking_duration_score']

df['smoking_score_int_score_3'] = (
    df['smoker_score'] * (df['ciggaretes_per_day_score'] ** df['smoking_duration_score']))

df['smoking_score_int_score_log'] = df['smoker_score'] ** df['ciggaretes_per_day_log']

In [105]:
smoke_features = ['smoker_score',
                  'ciggaretes_per_day_log', 'smoking_score_int_score',
                  'smoking_score_int_log', 'smoking_score_int_score_log',
                  'smoking_score_int_score_2', 'smoking_score_int_score_3']

In [106]:
check_corrs_with_target(df, col=smoke_features)

corrs_matrix_between:  ['smoker_score', 'ciggaretes_per_day_log', 'smoking_score_int_score', 'smoking_score_int_log', 'smoking_score_int_score_log', 'smoking_score_int_score_2', 'smoking_score_int_score_3']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
smoker_score,-0.090244,0.030081,-0.029108,-0.090927,-0.008845
ciggaretes_per_day_log,-0.057114,0.010710,0.003519,-0.088335,-0.026594
smoking_score_int_score,0.043129,-0.010952,0.056822,-0.024884,-0.022940
smoking_score_int_log,0.011216,0.006253,-0.000963,-0.038120,-0.022421
smoking_score_int_score_log,-0.057367,0.037104,-0.029631,-0.078990,-0.040684
smoking_score_int_score_2,-0.044030,0.019476,-0.007960,-0.071419,-0.031180
smoking_score_int_score_3,0.034131,-0.006945,0.086329,-0.011101,-0.010440


In [107]:
# Проверка на наличие пропущенных значений
df[smoke_features].isna().sum()

smoker_score                   0
ciggaretes_per_day_log         0
smoking_score_int_score        0
smoking_score_int_log          0
smoking_score_int_score_log    0
smoking_score_int_score_2      0
smoking_score_int_score_3      0
dtype: int64

### Колонка passive_smoking

In [108]:
df.passive_smoking.value_counts(dropna=False)

0    1215
1     378
Name: passive_smoking, dtype: int64

In [109]:
check_corrs_with_target(df, col='passive_smoking')

corrs_matrix_between:  passive_smoking


arterial_hypertension    0.011831
stroke                  -0.044094
IHD                      0.034347
heart_failure            0.028622
other_cardio_diseases   -0.001483
Name: passive_smoking, dtype: float64

### Колонка passive_smoking_frequency

In [110]:
df.passive_smoking_frequency.value_counts(dropna=False)

NaN                       1216
1-2 раза в неделю          158
4 и более раз в день        75
2-3 раза в день             59
не менее 1 раза в день      47
3-6 раз в неделю            38
Name: passive_smoking_frequency, dtype: int64

In [111]:
df.passive_smoking_frequency.fillna(0, inplace=True)

In [112]:
# Словарь для замены текста на порядок:
passive_smoking_dict = {'1-2 раза в неделю': 1,
                        '3-6 раз в неделю': 2,
                        'не менее 1 раза в день': 3,
                        '2-3 раза в день': 4,
                        '4 и более раз в день': 5}

In [113]:
df.passive_smoking_frequency.replace(passive_smoking_dict, inplace=True)

In [114]:
check_corrs_with_target(df, col='passive_smoking_frequency')

corrs_matrix_between:  passive_smoking_frequency


arterial_hypertension    0.003508
stroke                  -0.046057
IHD                      0.038805
heart_failure            0.052193
other_cardio_diseases    0.017315
Name: passive_smoking_frequency, dtype: float64

### Колонка alcohol

In [115]:
df.alcohol.value_counts(dropna=False)

употребляю в настоящее время    1046
никогда не употреблял            300
ранее употреблял                 247
Name: alcohol, dtype: int64

In [116]:
alcohol = pd.get_dummies(df.alcohol)
print('Before replace: ', list(alcohol.columns))

alcohol_col_map = {'никогда не употреблял': 'never_drink_alco',
                   'ранее употреблял': 'previous_drink_alco',
                   'употребляю в настоящее время': 'drink_alco_now'}

alcohol.rename(alcohol_col_map, axis=1, inplace=True)
print('After replace: ', list(alcohol.columns))

Before replace:  ['никогда не употреблял', 'ранее употреблял', 'употребляю в настоящее время']
After replace:  ['never_drink_alco', 'previous_drink_alco', 'drink_alco_now']


In [117]:
df = pd.concat([df, alcohol], axis=1)
df.drop('alcohol', axis=1, inplace=True)

In [118]:
alcohol_col = alcohol.columns
check_corrs_with_target(df, col=alcohol_col)

corrs_matrix_between:  Index(['never_drink_alco', 'previous_drink_alco', 'drink_alco_now'], dtype='object')


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
never_drink_alco,0.049645,-0.042137,0.015082,0.059077,0.049747
previous_drink_alco,0.063277,0.060122,0.094059,0.097195,0.019313
drink_alco_now,-0.089252,-0.013329,-0.085680,-0.123345,-0.054927


In [119]:
df['alcohol_score'] = df['never_drink_alco']*0 + \
    df['previous_drink_alco'] + df['drink_alco_now']*2

In [120]:
check_corrs_with_target(df, col='alcohol_score')

corrs_matrix_between:  alcohol_score


arterial_hypertension   -0.078958
stroke                   0.012484
IHD                     -0.059839
heart_failure           -0.104449
other_cardio_diseases   -0.057996
Name: alcohol_score, dtype: float64

### Колонка alcohol_duration

In [121]:
df.alcohol_duration.value_counts(dropna=False)

NaN     304
20.0    299
18.0    278
17.0    131
16.0    111
25.0     88
19.0     66
21.0     57
15.0     50
22.0     42
30.0     28
23.0     25
14.0     18
35.0     15
24.0     13
26.0      8
40.0      8
12.0      5
45.0      5
29.0      5
33.0      4
27.0      4
37.0      4
13.0      4
10.0      3
50.0      3
32.0      3
6.0       2
28.0      2
46.0      1
36.0      1
9.0       1
48.0      1
63.0      1
43.0      1
44.0      1
60.0      1
Name: alcohol_duration, dtype: int64

In [122]:
df['alcohol_duration_score'] = df['alcohol_duration'] // 5

In [123]:
df['alcohol_duration_log'] = np.log(df['alcohol_duration'] + 1)

In [124]:
df['alcohol_duration_score'] = np.where(
    df.alcohol_duration_score > 9, 9, df.alcohol_duration_score)

In [125]:
df.alcohol_duration.fillna(0, inplace=True)
df.alcohol_duration_score.fillna(0, inplace=True)
df.alcohol_duration_log.fillna(0, inplace=True)

In [126]:
df['alcohol_int_score_1'] = df['alcohol_score'] ** df['alcohol_duration_score']

df['alcohol_int_score_2'] = (
    df['alcohol_score'] * df['alcohol_duration_score'])

df['alcohol_int_score_3'] = df['alcohol_duration_log'] ** df['alcohol_score']

In [127]:
alco_features = ['alcohol_duration', 'alcohol_duration_score', 'alcohol_duration_log',
                 'alcohol_int_score_1', 'alcohol_int_score_2', 'alcohol_int_score_3']

In [128]:
check_corrs_with_target(df, col=alco_features)

corrs_matrix_between:  ['alcohol_duration', 'alcohol_duration_score', 'alcohol_duration_log', 'alcohol_int_score_1', 'alcohol_int_score_2', 'alcohol_int_score_3']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
alcohol_duration,0.006058,0.045679,0.005183,-0.013339,-0.033091
alcohol_duration_score,0.018031,0.047084,0.008138,-0.004915,-0.035852
alcohol_duration_log,-0.033025,0.046026,-0.007616,-0.043556,-0.045852
alcohol_int_score_1,0.019045,0.011645,-0.015414,-0.012578,-0.015989
alcohol_int_score_2,-0.017408,0.032432,-0.036184,-0.051155,-0.039593
alcohol_int_score_3,-0.062109,0.013989,-0.064165,-0.091707,-0.044619


### Колонка time_sleep_onset

In [129]:
df.time_sleep_onset = df.time_sleep_onset.apply(
    lambda x: dt.strptime(x, '%H:%M:%S'))

### ADD FEATURES about sleep onset

Примечание: время засыпания связано с развитием сердечно-сосудистых заболеваний о чем есть ряд научных публикаций, одна из них представлена ниже

Статья о влиянии времени засыпания на развитие сердечно-сосудистых заболеваний:
- https://doi.org/10.1093/ehjdh/ztab088

In [130]:
df['sleep_onset_early_22'] = np.where(
    df.time_sleep_onset.dt.time.between(time(20, 0), time(21, 59)), 1, 0)
df['sleep_onset_22'] = np.where(
    df.time_sleep_onset.dt.time.between(time(22, 0), time(22, 59)), 1, 0)
df['sleep_onset_23'] = np.where(
    df.time_sleep_onset.dt.time.between(time(23, 0), time(23, 59)), 1, 0)
df['sleep_onset_later_after_0'] = np.where(
    df.time_sleep_onset.dt.time.between(time(0, 0), time(4, 0)), 1, 0)

# additional:
df['early_onset'] = df['sleep_onset_early_22'] + df['sleep_onset_22']
df['lately_onset'] = df['sleep_onset_23'] + df['sleep_onset_later_after_0']

In [131]:
sleep_onset_cols = ['sleep_onset_early_22', 'sleep_onset_22',
                    'sleep_onset_23', 'sleep_onset_later_after_0', 'early_onset', 'lately_onset']

In [132]:
check_corrs_with_target(df, col=sleep_onset_cols)

corrs_matrix_between:  ['sleep_onset_early_22', 'sleep_onset_22', 'sleep_onset_23', 'sleep_onset_later_after_0', 'early_onset', 'lately_onset']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
sleep_onset_early_22,0.038944,0.038912,0.007070,0.010225,0.019608
sleep_onset_22,0.027138,0.064104,0.012710,-0.023735,0.012066
sleep_onset_23,-0.013236,-0.048355,0.004281,-0.002796,-0.034675
sleep_onset_later_after_0,-0.031067,-0.024444,-0.017935,0.019204,0.016151
early_onset,0.045152,0.079354,0.015401,-0.016712,0.021255
lately_onset,-0.047608,-0.078761,-0.014505,0.017477,-0.020490


### Колонка time_sleep_upset

In [133]:
df.time_sleep_upset = df.time_sleep_upset.apply(
    lambda x: dt.strptime(x, '%H:%M:%S'))

In [134]:
df['sleep_upset_early_6'] = np.where(
    df.time_sleep_upset.dt.time.between(time(4, 0), time(5, 59)), 1, 0)
df['sleep_upset_6'] = np.where(
    df.time_sleep_upset.dt.time.between(time(6, 0), time(6, 59)), 1, 0)
df['sleep_upset_7'] = np.where(
    df.time_sleep_upset.dt.time.between(time(7, 0), time(7, 59)), 1, 0)
df['sleep_upset_8'] = np.where(
    df.time_sleep_upset.dt.time.between(time(8, 0), time(8, 59)), 1, 0)
df['sleep_upset_after_9'] = np.where(
    df.time_sleep_upset.dt.time.between(time(9, 0), time(11, 59)), 1, 0)

# additional:
# Ранний подъем:
df['early_upset'] = df['sleep_upset_6'] + df['sleep_upset_7']

# Поздний подъем:
df['lately_upset'] = df['sleep_upset_8'] + df['sleep_upset_after_9']

In [135]:
upset_features = ['sleep_upset_early_6', 'sleep_upset_6', 'sleep_upset_7',
                  'sleep_upset_8', 'sleep_upset_after_9', 'early_upset', 'lately_upset']

In [136]:
df.sleep_upset_early_6.value_counts()

0    1398
1     195
Name: sleep_upset_early_6, dtype: int64

In [137]:
check_corrs_with_target(df, col=upset_features)

corrs_matrix_between:  ['sleep_upset_early_6', 'sleep_upset_6', 'sleep_upset_7', 'sleep_upset_8', 'sleep_upset_after_9', 'early_upset', 'lately_upset']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
sleep_upset_early_6,0.059959,-0.029609,-0.008345,0.039253,0.031984
sleep_upset_6,-0.035852,0.004285,0.007890,-0.013799,-0.012014
sleep_upset_7,-0.036765,0.031326,-0.008729,0.035565,-0.038354
sleep_upset_8,0.028764,-0.007423,0.038220,0.006858,0.013119
sleep_upset_after_9,0.009271,-0.023065,-0.025812,-0.070504,0.021491
early_upset,-0.068202,0.032787,-0.000377,0.019227,-0.046669
lately_upset,0.030296,-0.022269,0.013601,-0.043607,0.025885


### ADD FEATURE sleeptime

In [138]:
df['sleeptime'] = (df.time_sleep_upset -
                   df.time_sleep_onset).apply(lambda x: x.seconds//3600)

In [139]:
# Исправление дефекта при расчете времени сна из-за особенностей datetime
df['sleeptime'] = np.where(
    df['sleeptime'] > 14, df['sleeptime'] - 12, df['sleeptime'])

In [140]:
df['sleeptime'].value_counts()

7     457
8     453
6     230
9     214
10     93
5      67
11     32
4      22
12     11
3       8
0       4
13      2
Name: sleeptime, dtype: int64

### ADD FEATURE low_amout_sleep, high_amout_sleep

In [141]:
df['low_amount_sleep'] = np.where(df['sleeptime'] < 6, 1, 0)
df['high_amout_sleep'] = np.where(df['sleeptime'] > 9, 1, 0)

In [142]:
sleeptime_features = ['sleeptime', 'low_amount_sleep', 'high_amout_sleep']

In [143]:
check_corrs_with_target(df, col=sleeptime_features)

corrs_matrix_between:  ['sleeptime', 'low_amount_sleep', 'high_amout_sleep']


,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
sleeptime,0.035149,0.037473,-0.006052,-0.061437,-0.010208
low_amount_sleep,-0.013547,0.010021,-0.016293,0.058856,0.056018
high_amout_sleep,0.024350,0.024505,-0.015854,-0.031118,0.042978


In [144]:
df.drop(['time_sleep_onset', 'time_sleep_upset'], axis=1, inplace=True)

### Колонка midday_sleep

In [145]:
df.midday_sleep.value_counts(dropna=False)

0    1232
1     361
Name: midday_sleep, dtype: int64

In [146]:
check_corrs_with_target(df, col='midday_sleep')

corrs_matrix_between:  midday_sleep


arterial_hypertension    0.075877
stroke                   0.070712
IHD                      0.049903
heart_failure            0.002388
other_cardio_diseases   -0.056408
Name: midday_sleep, dtype: float64

### Колонка sport

In [147]:
df.sport.value_counts(dropna=False)

0    1491
1     102
Name: sport, dtype: int64

In [148]:
check_corrs_with_target(df, col='sport')

corrs_matrix_between:  sport


arterial_hypertension   -0.011300
stroke                   0.024806
IHD                      0.000465
heart_failure            0.020270
other_cardio_diseases    0.031178
Name: sport, dtype: float64

### Колонка religion_clubs

In [149]:
df.religion_clubs.value_counts(dropna=False)

0    1557
1      36
Name: religion_clubs, dtype: int64

In [150]:
check_corrs_with_target(df, col='religion_clubs')

corrs_matrix_between:  religion_clubs


arterial_hypertension   -0.031819
stroke                  -0.032523
IHD                     -0.014801
heart_failure            0.018303
other_cardio_diseases    0.049226
Name: religion_clubs, dtype: float64

## Подготовка данных

In [151]:
# Проверка на наличие пропусков в итоговом наборе данных
df.isna().sum().sum()

0

### ADD FEATURE cardio_score

Примечание: существует довольно много разных шкал риска, но не все они легко применимы на данном датасете. В данном случае применен измененный Jakarta Cardiovascular Score (посчитан из того, что было в датасете)

https://www.researchgate.net/publication/269624976/figure/tbl4/AS:669948162629639@1536739497178/The-Framingham-Score-and-the-Jakarta-Cardiovascular-Score.png

In [153]:
df['cardio_score'] = df['sex'] + df['smoke'] * 2 + df['previous_smoke'] * 1 + \
    df['diabetes'] * 2 - df['sport'] * 3 + \
    df['retired'] * 3 + df['passive_smoking']

In [154]:
check_corrs_with_target(df, col='cardio_score')

corrs_matrix_between:  cardio_score


arterial_hypertension    0.200888
stroke                   0.067760
IHD                      0.203711
heart_failure            0.100453
other_cardio_diseases   -0.002081
Name: cardio_score, dtype: float64

In [155]:
train = df[df['train'] == 1].drop('train', axis=1)
X_test = df[df['train'] == 0].drop('train', axis=1)

# В ходе экспериментов произошел сознательный отказ от использования StandardScaler (и других скейлеров) в пользу интерпретируемости модели

### Так как данные имеют большую суммарную связь с артериальной гипертнезией, то сосредоточимся на предсказании этого показателя. 
#### В дальнейшем обучив модель и "подставив" предсказания в тестовый датасет продолжим предсказывать другие

In [156]:
target_train.corr()

,arterial_hypertension,stroke,IHD,heart_failure,other_cardio_diseases
arterial_hypertension,1.000000,0.122720,0.296755,0.266396,0.086785
stroke,0.122720,1.000000,0.015392,-0.002087,-0.030536
IHD,0.296755,0.015392,1.000000,0.522954,0.049801
heart_failure,0.266396,-0.002087,0.522954,1.000000,0.138133
other_cardio_diseases,0.086785,-0.030536,0.049801,0.138133,1.000000


#### Исходя из корреляций между АГ и другими болезнями у метода хорошие перспективы

# Поиск лучшей метрики с помощью логистической регрессии

In [157]:
# Гиперпараметры логистической регрессии для перебора с помощью GridsearchCV
C = [1, 0.1, 0.01, 0.001]
iters = [50, 100, 200, 300]
epsilon_stop = [0.0001]

param_grid_logreg = [
    {'C': C,
     'penalty': ['l1'],
     'solver': ['saga', 'liblinear'],
     'class_weight':[None, 'balanced'],
     'multi_class': ['auto', 'ovr'],
     'max_iter': iters,
     'tol': epsilon_stop},
    {'C': C,
     'penalty': ['l2'],
     'solver': ['newton-cg', 'liblinear', 'lbfgs', 'saga'],
     'class_weight':[None, 'balanced'],
     'multi_class': ['auto', 'ovr'],
     'max_iter': iters,
     'tol':epsilon_stop},
    {'penalty': ['none'],
     'solver': ['saga'],
     'class_weight':[None, 'balanced'],
     'multi_class': ['auto', 'ovr'],
     'max_iter': iters,
     'tol':epsilon_stop}]

In [158]:
def score_model(model, params=None, oversampler=None, cv=None, X_train=None, y_train=None, n_splits=5, random_state=None):
    """
    Creates folds manually, and upsamples within each fold.
    """

    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)

    if cv is None:
        cv = StratifiedKFold(n_splits=n_splits)

    thresholds = []
    recall_macro_scores = []
    f1_scores = []
    

    for train_fold_index, val_fold_index in cv.split(X_train, y_train):
        # Определение тренировочного набора
        X_train_fold, y_train_fold = X_train.iloc[train_fold_index], y_train[train_fold_index]
        # Определение тестового набора
        X_val_fold, y_val_fold = X_train.iloc[val_fold_index], y_train[val_fold_index]

        # Балансировка только тренировочного датасета
        X_train_fold_upsample, y_train_fold_upsample = oversampler.fit_resample(X_train_fold,
                                                                                y_train_fold)
        # Обучени модели на сбалансированном тренировочном датасете
        model_obj = model.fit(X_train_fold_upsample, y_train_fold_upsample)

        # Оценка модели на несбалансированном, тестовом датасете
        probs = model_obj.predict_proba(X_val_fold)
        threshold = find_opt_threshold(y_val_fold, probs)
        thresholds.append(threshold)
        y_pred = np.where(probs[:, 1] > threshold, 1, 0)

        recall_macro_score = recall_score(y_val_fold, y_pred, average='macro')
        recall_macro_scores.append(recall_macro_score)

        f1score = f1_score(y_val_fold, y_pred)
        f1_scores.append(f1score)

    print('recall_macro_scores: ', recall_macro_scores)
    print('recall_macro_mean: ', np.mean(recall_macro_scores))
    print()
    print('f1_scores: ', f1_scores)
    print('f1_mean: ', np.mean(f1_scores))
    print()
    print('mean_threshold: ', np.mean(thresholds))
    # Функция возвращает среднее пороговое значнием для максимальной метрики
    return np.mean(thresholds)

In [159]:
# Модель логистической регрессии с балансировкой выборки.
def fit_model(df, target, model='logreg', scoring='f1',
              split=True,
              reduce_corrs=False, threshold_reduce=0.05,
              grid=False, oversampling=False,  param_grid=None,
              oversampler='random', resample_strategy='auto',
              return_predicted=False, X_test=None,
              random_state=None, n_splits=5):

    cv_grid = StratifiedKFold(random_state=random_state)

    if reduce_corrs == True:
        df, X_test = reduce_low_corrs_with_target(df=df, target=target,
                                                  threshold=threshold_reduce, return_predicted=return_predicted,
                                                  X_test=X_test)

    used_features = df.columns
    print('Used features: ', list(df.columns))
    print()

    X = df.drop(target, axis=1)
    y = df[target]

    if split == True:
        X_train, X_valid, y_train, y_valid = train_test_split(
            X, y, test_size=0.20, stratify=y, random_state=random_state)
    else:
        X_train = X.copy()
        y_train = y.copy()

    if model == 'logreg':
        # Логистическая регрессия
        clf = LogisticRegression(random_state=random_state, n_jobs=-1)

        # Поиск гиперпараметров GridSearchCV при помощи перебора параметров по сетке param_grid:
        if grid == True:
            grid_search = GridSearchCV(
                clf, param_grid, scoring=scoring, n_jobs=-1, cv=cv_grid)
            grid_search.fit(X, y)

        # Печатаем параметры развернуто:
            best_model = grid_search.best_estimator_
            best_parameters = best_model.get_params()
            for param_name in sorted(best_parameters.keys()):
                print('\t%s: %r' % (param_name, best_parameters[param_name]))

        clf = LogisticRegression(**best_parameters)

    print()

    # Балансировка выборки
    if oversampling == True:
        if oversampler == 'random':
            oversampler = RandomOverSampler(
                sampling_strategy=resample_strategy, random_state=random_state)

        if oversampler == 'smote':
            oversampler = SMOTE(
                sampling_strategy=resample_strategy, random_state=random_state)

        threshold = score_model(model=clf, params=best_parameters, oversampler=oversampler,
                                cv=None, X_train=X_train, y_train=y_train, n_splits=n_splits, random_state=random_state)

        X_train, y_train = oversampler.fit_resample(X_train, y_train)

    clf.fit(X_train, y_train)

    coefs_df = pd.DataFrame(data=clf.coef_[0],
                            index=X_train.columns,
                            columns=['coef'])
    print()
    print('Coefficients of LogisticRegression:')
    print(coefs_df.sort_values(by='coef', ascending=False))
    print()

    print('Odds:')
    odds = np.exp(clf.coef_[0])
    odds_df = pd.DataFrame(odds,
                           X.columns,
                           columns=['coef'])\
        .sort_values(by='coef', ascending=False)
    print(odds_df)
    print()

    if split == True:
        probs = clf.predict_proba(X_valid)
        threshold = find_opt_threshold(y_valid, probs)

        y_pred = np.where(probs[:, 1] > threshold, 1, 0)
        print()
        print('valid_recall_score: ', recall_score(
            y_valid, y_pred, average='macro'))
    else:
        probs = clf.predict_proba(X)
        threshold = find_opt_threshold(y, probs)
        print('Self threshold: ', threshold)

        y_pred = np.where(probs[:, 1] > threshold, 1, 0)
        print()
        print('self_recall_score: ', recall_score(y, y_pred, average='macro'))

    if return_predicted == True:
        probs_test = clf.predict_proba(X_test)[:, 1]
        predicted_test = np.where(probs_test > threshold, 1, 0)
        return predicted_test
    else:
        return clf, threshold

In [160]:
df_ah = pd.concat([train, target_train['arterial_hypertension']], axis=1)

In [161]:
ah = MultiCollinearityEliminator(
    df_ah, 'arterial_hypertension', 0.6).autoEliminateMulticollinearity()

In [162]:
SEED = None

In [163]:
X_pred = pd.DataFrame()

In [164]:
ah.arterial_hypertension.value_counts()

0    509
1    446
Name: arterial_hypertension, dtype: int64

In [165]:
ah_pred = fit_model(ah, 'arterial_hypertension',
                    split=False,
                    reduce_corrs=True, threshold_reduce=0.06,
                    model='logreg', grid=True, param_grid=param_grid_logreg, scoring='f1',
                    oversampling=True, oversampler='random', resample_strategy='auto',
                    return_predicted=True, X_test=X_test,
                    random_state=SEED, n_splits=5)

Used features:  ['crafters_other', 'household', 'farmer/fisherman', 'chronic_lung_disease', 'early_upset', 'midday_sleep', 'married', 'drink_alco_now', 'oncology', 'bronchial_asthma', 'hard_industry', 'smoke', 'fractures', 'unknown_id_feature', 'education', 'sex', 'comorbid_count', 'widowed', 'diabetes', 'retired', 'intake_medicines', 'arterial_hypertension']

	C: 0.1
	class_weight: None
	dual: False
	fit_intercept: True
	intercept_scaling: 1
	l1_ratio: None
	max_iter: 50
	multi_class: 'auto'
	n_jobs: -1
	penalty: 'l1'
	random_state: None
	solver: 'saga'
	tol: 0.0001
	verbose: 0
	warm_start: False

Threshold=0.614, Recall_macro=0.72505
Threshold=0.341, Recall_macro=0.73017
Threshold=0.533, Recall_macro=0.74631
Threshold=0.554, Recall_macro=0.75110
Threshold=0.518, Recall_macro=0.77239
recall_macro_scores:  [0.7250495703899538, 0.7301718440185063, 0.7463097598589998, 0.7511015642211941, 0.7723872387238724]
recall_macro_mean:  0.7450039954425053

f1_scores:  [0.7011494252873562, 0.738916

In [166]:
print('len of predicted: ', sum(ah_pred))
X_pred['arterial_hypertension'] = ah_pred
X_test['arterial_hypertension'] = ah_pred

len of predicted:  314


In [167]:
df_ihd = pd.concat(
    [train, target_train[['arterial_hypertension', 'IHD']]], axis=1)

In [168]:
ihd = MultiCollinearityEliminator(
    df_ihd, 'IHD', 0.6).autoEliminateMulticollinearity()

In [169]:
ihd['IHD'].value_counts()

0    838
1    117
Name: IHD, dtype: int64

In [170]:
ihd_pred = fit_model(ihd, 'IHD', split=False,
                     reduce_corrs=True, threshold_reduce=0.04,
                     model='logreg', grid=True, param_grid=param_grid_logreg, scoring='f1',
                     oversampling=True, oversampler='random', resample_strategy='auto',
                     return_predicted=True, X_test=X_test, random_state=SEED, n_splits=3)

Used features:  ['divorced', 'oncology', 'bronchial_asthma', 'midday_sleep', 'education', 'fracture_on_retire', 'top_management', 'is_russian', 'service', 'household', 'farmer/fisherman', 'fractures', 'tuberculosis', 'stop_work_due_disease', 'smoking_score_int_score_3', 'widowed', 'previous_drink_alco', 'is_comorbid', 'diabetes', 'intake_medicines', 'retired', 'arterial_hypertension', 'IHD']

	C: 1
	class_weight: 'balanced'
	dual: False
	fit_intercept: True
	intercept_scaling: 1
	l1_ratio: None
	max_iter: 200
	multi_class: 'ovr'
	n_jobs: -1
	penalty: 'l1'
	random_state: None
	solver: 'liblinear'
	tol: 0.0001
	verbose: 0
	warm_start: False

Threshold=0.381, Recall_macro=0.73136
Threshold=0.423, Recall_macro=0.74773
Threshold=0.404, Recall_macro=0.74593
recall_macro_scores:  [0.7313644688644688, 0.7477253928866832, 0.7459332781913427]
recall_macro_mean:  0.7416743799808315

f1_scores:  [0.3803680981595092, 0.38202247191011235, 0.37988826815642457]
f1_mean:  0.3807596127420154

mean_thres

In [171]:
X_pred['IHD'] = ihd_pred
X_test['IHD'] = ihd_pred
#target_train['IHD'] = ihd_train
ihd_pred.sum()

301

In [172]:
df_hf = pd.concat(
    [train, target_train[['arterial_hypertension', 'IHD', 'heart_failure']]], axis=1)

In [173]:
hf = MultiCollinearityEliminator(
    df_hf, 'heart_failure', 0.6).autoEliminateMulticollinearity()

In [174]:
hf['heart_failure'].value_counts()

0    859
1     96
Name: heart_failure, dtype: int64

In [175]:
hf_pred = fit_model(hf, 'heart_failure',
                    reduce_corrs=True, threshold_reduce=0.04, split=False,
                    model='logreg', scoring='f1', grid=True, param_grid=param_grid_logreg,
                    oversampling=True, oversampler='random', resample_strategy='auto',
                    return_predicted=True, X_test=X_test, n_splits=3,
                    random_state=SEED)

Used features:  ['lately_upset', 'oncology', 'previous_smoke', 'top_management', 'married', 'crafters_other', 'low_qualified', 'tuberculosis', 'chronic_lung_disease', 'cohabited', 'passive_smoking_frequency', 'stop_work_due_disease', 'id_2', 'fractures', 'low_amount_sleep', 'farmer/fisherman', 'id_4', 'sleeptime', 'sleep_upset_after_9', 'education', 'sex', 'never_smoke', 'household', 'diabetes', 'bronchial_asthma', 'comorbid_count', 'drink_alco_now', 'widowed', 'job', 'intake_medicines', 'arterial_hypertension', 'IHD', 'heart_failure']

	C: 0.01
	class_weight: 'balanced'
	dual: False
	fit_intercept: True
	intercept_scaling: 1
	l1_ratio: None
	max_iter: 50
	multi_class: 'auto'
	n_jobs: -1
	penalty: 'l1'
	random_state: None
	solver: 'saga'
	tol: 0.0001
	verbose: 0
	warm_start: False

Threshold=0.488, Recall_macro=0.84713
Threshold=0.405, Recall_macro=0.76541
Threshold=0.426, Recall_macro=0.74279
recall_macro_scores:  [0.8471254355400697, 0.7654064685314685, 0.7427884615384616]
recall_mac

In [176]:
X_pred['heart_failure'] = hf_pred
X_test['heart_failure'] = hf_pred
#target_train['heart_failure'] = hf_train
sum(hf_pred)

303

# Предсказание ACD

In [177]:
df_stroke = pd.concat([train, target_train[[
                      'arterial_hypertension', 'IHD', 'heart_failure', 'stroke']]], axis=1)

In [178]:
stroke = MultiCollinearityEliminator(
    df_stroke, 'stroke', 0.6).autoEliminateMulticollinearity()

In [179]:
stroke['stroke'].value_counts()

0    914
1     41
Name: stroke, dtype: int64

In [180]:
stroke_pred = fit_model(stroke, 'stroke', split=False,
                        reduce_corrs=True, threshold_reduce=0.03,
                        model='logreg', scoring='f1', grid=True, param_grid=param_grid_logreg,
                        oversampling=True, oversampler='random', resample_strategy='auto',
                        return_predicted=True, X_test=X_test, random_state=SEED, n_splits=3)

Used features:  ['tuberculosis', 'sleep_upset_7', 'religion_clubs', 'early_upset', 'european', 'sleeptime', 'sleep_onset_early_22', 'top_management', 'id_3', 'diabetes', 'bronchial_asthma', 'junior_specialists', 'single', 'passive_smoking_frequency', 'alcohol_duration_score', 'sleep_onset_23', 'christian', 'farmer/fisherman', 'previous_smoke', 'smoke', 'previous_drink_alco', 'education', 'cardio_score', 'fractures', 'midday_sleep', 'sex', 'job', 'early_onset', 'id_1', 'stop_work_due_disease', 'crafters_other', 'intake_medicines', 'arterial_hypertension', 'stroke']

	C: 1
	class_weight: 'balanced'
	dual: False
	fit_intercept: True
	intercept_scaling: 1
	l1_ratio: None
	max_iter: 200
	multi_class: 'ovr'
	n_jobs: -1
	penalty: 'l2'
	random_state: None
	solver: 'saga'
	tol: 0.0001
	verbose: 0
	warm_start: False

Threshold=0.101, Recall_macro=0.66956
Threshold=0.718, Recall_macro=0.70038
Threshold=0.058, Recall_macro=0.68092
recall_macro_scores:  [0.6695550351288055, 0.7003783102143758, 0.68

In [181]:
X_pred['stroke'] = stroke_pred
X_test['stroke'] = stroke_pred
sum(stroke_pred)

202

## Предсказание other

In [182]:
df_other = pd.concat([train, target_train[['arterial_hypertension',
                                           'IHD', 'heart_failure', 'stroke', 'other_cardio_diseases']]], axis=1)

In [183]:
other = MultiCollinearityEliminator(
    df_other, 'other_cardio_diseases', 0.6).autoEliminateMulticollinearity()

In [184]:
other['other_cardio_diseases'].value_counts()

0    869
1     86
Name: other_cardio_diseases, dtype: int64

In [185]:
other_pred = fit_model(other, 'other_cardio_diseases', split=False,
                       reduce_corrs=True, threshold_reduce=0.03,
                       model='logreg', scoring='f1', grid=True, param_grid=param_grid_logreg,
                       oversampling=True, oversampler='random', resample_strategy='auto',
                       return_predicted=True, X_test=X_test, random_state=SEED, n_splits=2)

Used features:  ['stroke', 'sport', 'stop_work_due_disease', 'service', 'sleep_upset_early_6', 'widowed', 'single', 'diabetes', 'sleep_onset_23', 'fractures', 'office', 'sleep_upset_7', 'trauma_last_year', 'trauma_on_retire', 'top_management', 'ciggaretes_per_day_score', 'high_amout_sleep', 'early_upset', 'religion_clubs', 'senior_specialist', 'IHD', 'european', 'id_2', 'oncology', 'low_amount_sleep', 'christian', 'midday_sleep', 'low_qualified', 'alcohol_score', 'job', 'chronic_lung_disease', 'is_comorbid', 'sex', 'arterial_hypertension', 'hard_industry', 'intake_medicines', 'heart_failure', 'other_cardio_diseases']

	C: 0.1
	class_weight: 'balanced'
	dual: False
	fit_intercept: True
	intercept_scaling: 1
	l1_ratio: None
	max_iter: 50
	multi_class: 'auto'
	n_jobs: -1
	penalty: 'l2'
	random_state: None
	solver: 'liblinear'
	tol: 0.0001
	verbose: 0
	warm_start: False

Threshold=0.452, Recall_macro=0.67541
Threshold=0.455, Recall_macro=0.67897
recall_macro_scores:  [0.675407645014702, 0.

In [186]:
X_pred['other_cardio_diseases'] = other_pred
X_test['other_cardio_diseases'] = other_pred
sum(other_pred)

337

In [187]:
result = X_pred[['arterial_hypertension', 'stroke', 'IHD',
                 'heart_failure', 'other_cardio_diseases']].copy()

In [188]:
submit = pd.read_csv('data/sample_solution.csv')

In [189]:
submit.iloc[:, 1:] = result.values

In [190]:
submit.iloc[:, 1:].sum()

Артериальная гипертензия              314
ОНМК                                  202
Стенокардия, ИБС, инфаркт миокарда    301
Сердечная недостаточность             303
Прочие заболевания сердца             337
dtype: int64

In [191]:
submit.to_csv('submits/submit_screencast.csv', index=False)

In [ ]:
›